# FBI background checks - States and categories

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import datetime as dt
import numpy as np

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
month_year_updated = dt.date.today().strftime("%m_%Y")

---

### Bless you @jsvine

In [5]:
url = "https://raw.githubusercontent.com/BuzzFeedNews/nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv"

### Read data

In [6]:
src = pd.read_csv(url)

In [7]:
src.iloc[0]

month                        2021-11
state                        Alabama
permit                       24489.0
permit_recheck                 229.0
handgun                      22126.0
long_gun                     21230.0
other                         1319.0
multiple                        1265
admin                            0.0
prepawn_handgun                 15.0
prepawn_long_gun                11.0
prepawn_other                    2.0
redemption_handgun            2162.0
redemption_long_gun           1202.0
redemption_other                14.0
returned_handgun                39.0
returned_long_gun                0.0
returned_other                   0.0
rentals_handgun                  0.0
rentals_long_gun                 0.0
private_sale_handgun            23.0
private_sale_long_gun           26.0
private_sale_other               4.0
return_to_seller_handgun         2.0
return_to_seller_long_gun        0.0
return_to_seller_other           0.0
totals                         74158
N

In [8]:
src["year"] = src["month"].str[:4]

### Exclude permits figures

In [9]:
src["totals_no_permits"] = src[["handgun", "other", "multiple", "long_gun"]].sum(axis=1)

In [10]:
src["year"] = src["month"].str[:4]

In [11]:
df = src[
    [
        "month",
        "year",
        "state",
        "handgun",
        "other",
        "multiple",
        "long_gun",
        "totals_no_permits",
    ]
]

In [12]:
df.columns

Index(['month', 'year', 'state', 'handgun', 'other', 'multiple', 'long_gun',
       'totals_no_permits'],
      dtype='object')

In [13]:
grouped = (
    df[df["year"] > "2008"]
    .groupby(["year"])
    .agg(
        {
            "handgun": sum,
            "long_gun": sum,
            "other": sum,
            "multiple": sum,
            "totals_no_permits": sum,
        }
    )
    .reset_index()
)
grouped

,year,handgun,long_gun,other,multiple,totals_no_permits
0,2009,3689604.0,4978641.0,23208.0,235685,8927138.0
1,2010,3678001.0,4842184.0,52761.0,180609,8753555.0
2,2011,4301389.0,5445966.0,70137.0,219618,10037110.0
3,2012,5682963.0,6866225.0,162823.0,230270,12942281.0
4,2013,6387502.0,7128798.0,212776.0,241360,13970436.0
5,2014,6199243.0,5543371.0,268263.0,225475,12236352.0
6,2015,7333808.0,5479441.0,329499.0,242375,13385123.0
7,2016,8085498.0,5988511.0,586137.0,257723,14917869.0
8,2017,7226979.0,5234757.0,399993.0,236167,13097896.0
9,2018,6576111.0,4916533.0,494299.0,257088,12244031.0


In [14]:
round((grouped.iloc[11, 5] - grouped.iloc[10, 5]) / grouped.iloc[10, 5] * 100, 0)

64.0

In [15]:
grouped.iloc[11, 5]

20279035.0

In [16]:
grouped["other/mulitple"] = grouped["multiple"] + grouped["other"]

In [17]:
grouped.columns

Index(['year', 'handgun', 'long_gun', 'other', 'multiple', 'totals_no_permits',
       'other/mulitple'],
      dtype='object')

In [18]:
grouped[["year", "handgun", "long_gun", "other/mulitple"]].to_csv(
    "data/processed/total_by_year.csv", index=False
)

In [19]:
exclude = [
    "Virgin Islands",
    "Guam",
    "Puerto Rico",
    "Mariana Islands",
    "District of Columbia",
]

In [20]:
grouped_states = (
    df[df["year"] > "2008"]
    .groupby(["year", "state"])
    .agg({"handgun": sum, "long_gun": sum, "totals_no_permits": sum})
    .reset_index()
)

In [21]:
change_states_2019_2020 = (
    grouped_states[
        (grouped_states["year"] > "2018") & (grouped_states["year"] < "2021")
    ]
    .pivot(index="state", columns="year", values="totals_no_permits")
    .reset_index()
)

In [22]:
change_states_2019_2020["change"] = round(
    (change_states_2019_2020["2020"] - change_states_2019_2020["2019"])
    / change_states_2019_2020["2019"]
    * 100,
    2,
)

In [23]:
state_change_export = change_states_2019_2020[
    ~change_states_2019_2020["state"].isin(exclude)
].sort_values("change", ascending=False)

In [24]:
state_change_export.to_csv("data/processed/state_change_2019-2020.csv", index=False)

In [25]:
state_change_export

year,state,2019,2020,change
24,Michigan,253528.0,716416.0,182.58
0,Alabama,297308.0,608531.0,104.68
47,Utah,89051.0,179224.0,101.26
10,Georgia,280074.0,562732.0,100.92
2,Arizona,274127.0,544842.0,98.76
22,Maryland,103887.0,204071.0,96.44
32,New Jersey,89774.0,176059.0,96.11
42,Rhode Island,22858.0,44292.0,93.77
30,Nevada,96680.0,177884.0,83.99
7,Delaware,39723.0,70245.0,76.84
